1. 读取 transformer doc source，提取 target code
2. 通过 gpt 处理为函数 + 测试用例，搞 100 个，两种设置：
    a. 函数名 + 函数注释 => 补全函数
    b. 函数名 + 函数注释 + 步骤注释 => 补全步骤

In [1]:
import yaml

def load_doc_tree(path):
    # Open the YAML file and load its contents
    with open(yaml_file_path, "r") as yaml_file:
        # Load YAML data
        yaml_data = yaml.load(yaml_file, Loader=yaml.FullLoader)
        return yaml_data
    
base_path = "./transformers/docs/source/en"
yaml_file_path = f"{base_path}/_toctree.yml"
yaml_data = load_doc_tree(yaml_file_path)
yaml_data

[{'sections': [{'local': 'index', 'title': '🤗 Transformers'},
   {'local': 'quicktour', 'title': 'Quick tour'},
   {'local': 'installation', 'title': 'Installation'}],
  'title': 'Get started'},
 {'sections': [{'local': 'pipeline_tutorial',
    'title': 'Run inference with pipelines'},
   {'local': 'autoclass_tutorial',
    'title': 'Write portable code with AutoClass'},
   {'local': 'preprocessing', 'title': 'Preprocess data'},
   {'local': 'training', 'title': 'Fine-tune a pretrained model'},
   {'local': 'run_scripts', 'title': 'Train with a script'},
   {'local': 'accelerate',
    'title': 'Set up distributed training with 🤗 Accelerate'},
   {'local': 'peft', 'title': 'Load and train adapters with 🤗 PEFT'},
   {'local': 'model_sharing', 'title': 'Share your model'},
   {'local': 'transformers_agents', 'title': 'Agents'},
   {'local': 'llm_tutorial', 'title': 'Generation with LLMs'}],
  'title': 'Tutorials'},
 {'sections': [{'isExpanded': False,
    'sections': [{'local': 'tasks/seq

In [3]:
def get_all_path_from_yaml(doc):
    all_path = []
    
    if type(doc) == list:
        for section in doc:
            all_path += get_all_path_from_yaml(section)
            
    if type(doc) == dict:
        if 'sections' in doc:
            all_path += get_all_path_from_yaml(doc['sections'])
            
        if 'local' in doc:
            all_path.append(doc['local'])
        
    return all_path
            
all_path = get_all_path_from_yaml(yaml_data)
len(all_path), all_path

(356,
 ['index',
  'quicktour',
  'installation',
  'pipeline_tutorial',
  'autoclass_tutorial',
  'preprocessing',
  'training',
  'run_scripts',
  'accelerate',
  'peft',
  'model_sharing',
  'transformers_agents',
  'llm_tutorial',
  'tasks/sequence_classification',
  'tasks/token_classification',
  'tasks/question_answering',
  'tasks/language_modeling',
  'tasks/masked_language_modeling',
  'tasks/translation',
  'tasks/summarization',
  'tasks/multiple_choice',
  'tasks/audio_classification',
  'tasks/asr',
  'tasks/image_classification',
  'tasks/semantic_segmentation',
  'tasks/video_classification',
  'tasks/object_detection',
  'tasks/zero_shot_object_detection',
  'tasks/zero_shot_image_classification',
  'tasks/monocular_depth_estimation',
  'tasks/image_to_image',
  'tasks/knowledge_distillation_for_image_classification',
  'tasks/image_captioning',
  'tasks/document_question_answering',
  'tasks/visual_question_answering',
  'tasks/text-to-speech',
  'generation_strategie

In [25]:
def iter_markdown_line(path):
    with open(path, encoding="utf-8") as f:
        for l in f:
            yield l

def iter_block(path):
    block = []
    for l in iter_markdown_line(path):
        # before
        if l[0] == '#':
            yield block
            block = []
            
        if "```py\n" in l:
            yield block
            block = []
            
        block.append(l)
        
        # after
        if "```\n" in l:
            yield block
            block = []
    
def print_block(b):
    for l in b:
        print(l, end="")
    
    print("--------------------------------------------------------")

def get_valid_blocks(path):
    all_blocks = [b for b in iter_block(path)]
    valid_blocks = []

    for idx, block in enumerate(all_blocks):
        if len(block) == 0:
            continue
        if "```py" in block[0]:
            res = []
            if idx > 0:
                res += all_blocks[idx-1] # attach previous block
            res += block
            
            valid_blocks.append(res)
            
    return valid_blocks

path = f"{base_path}/quicktour.md"

for block in get_valid_blocks(path):
    print_block(block)

## Pipeline

<Youtube id="tiZFewofSLM"/>

The [`pipeline`] is the easiest and fastest way to use a pretrained model for inference. You can use the [`pipeline`] out-of-the-box for many tasks across different modalities, some of which are shown in the table below:

<Tip>

For a complete list of available tasks, check out the [pipeline API reference](./main_classes/pipelines).

</Tip>

| **Task**                     | **Description**                                                                                              | **Modality**    | **Pipeline identifier**                       |
|------------------------------|--------------------------------------------------------------------------------------------------------------|-----------------|-----------------------------------------------|
| Text classification          | assign a label to a given sequence of text                                                                   | NLP             | pipeline(task=“sentiment-analysi

In [26]:
all_valid_blocks = []

for path in all_path:
    for block in get_valid_blocks(f"{base_path}/{path}.md"):
        all_valid_blocks.append(block)
        
        if len(all_valid_blocks) % 10 == 0:
            print("len = ", len(all_valid_blocks))
            print_block(block)
            
len(all_valid_blocks) # 896 valid blocks

len =  10
</pt>
<tf>
Use [`TFAutoModelForSequenceClassification`] and [`AutoTokenizer`] to load the pretrained model and it's associated tokenizer (more on an `TFAutoClass` in the next section):

```py
>>> from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

>>> model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
>>> tokenizer = AutoTokenizer.from_pretrained(model_name)
```
--------------------------------------------------------
len =  20

<Tip>

See the [task summary](./task_summary) for tasks supported by an [`AutoModel`] class.

</Tip>

Now pass your preprocessed batch of inputs directly to the model. You can pass the tensors as-is:

```py
>>> tf_outputs = tf_model(tf_batch)
```
--------------------------------------------------------
len =  30
</pt>
<tf>
Create a model from your custom configuration with [`TFAutoModel.from_config`]:

```py
>>> from transformers import TFAutoModel

>>> my_model = TFAutoModel.from_config(my_config)
```
-

896

In [79]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

def get_chat_model():
    BASE_URL = "https://autoagents-global.openai.azure.com"
    API_KEY = "6c1c61bd992146a1bbcde4a80fef51ba"
    model = AzureChatOpenAI(
        temperature=0.5,
        openai_api_base=BASE_URL,
        openai_api_version="2023-08-01-preview",
        deployment_name="gpt-35-turbo-16k",
        openai_api_key=API_KEY,
        openai_api_type="azure",
    )
    return model
    
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

def valid_block_to_std_block(block):
    response_schemas = [
        ResponseSchema(name="func_name", description="function name"),
        ResponseSchema(name="func_import", description="package need to import to support the function"),
        ResponseSchema(name="func_def", description="definition of the function"),
        ResponseSchema(name="func_comment", description="comment of function, include params and return description"),
        ResponseSchema(name="func_impl", description="implement of function, include comment of each step, with return value"),
        ResponseSchema(name="func_whole", description="whole function include all parts: import, definition, params, comments, implementation, return value"),
        ResponseSchema(name="func_test", description="test function with 3-5 test case with test entry function. use assert to test the function.")
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    
    format_instructions = output_parser.get_format_instructions()
    prompt = ChatPromptTemplate(
        messages=[
            HumanMessagePromptTemplate.from_template("{format_instructions}\nBased on the instruction and example code provided, help me generate python code.\n{block}")
        ],
        input_variables=["block"],
        partial_variables={"format_instructions": format_instructions}
    )
    
    model = get_chat_model()
    
    _input = prompt.format_prompt(block=block)
    output = model(_input.to_messages())
    
    try:
        response = output_parser.parse(output.content)
        return response
    except:
        return output.content

std_block = valid_block_to_std_block("".join(all_valid_blocks[1]))
print(std_block)

{'func_name': 'classifier', 'func_import': 'from transformers import pipeline', 'func_def': 'def classifier(text: str) -> List[Dict[str, Union[str, float]]]:', 'func_comment': 'Performs sentiment analysis on the input text.\n\nArgs:\n    text (str): The input text to analyze.\n\nReturns:\n    List[Dict[str, Union[str, float]]]: A list of dictionaries containing the sentiment label and score.\n', 'func_impl': "result = pipeline('sentiment-analysis')(text)\nreturn result", 'func_whole': 'from transformers import pipeline\n\ndef classifier(text: str) -> List[Dict[str, Union[str, float]]]:\n    """Performs sentiment analysis on the input text.\n\n    Args:\n        text (str): The input text to analyze.\n\n    Returns:\n        List[Dict[str, Union[str, float]]]: A list of dictionaries containing the sentiment label and score.\n    """\n    result = pipeline(\'sentiment-analysis\')(text)\n    return result', 'func_test': "def test_classifier():\n    assert classifier('We are very happy to 

In [80]:
def get_code(block):
    if "```" in block and "func_name" in block:        
        json_data = block
        json_data.split("\n")[1:-2]
        d = eval(json_data)
    else:
        d = block
    return d['func_name'], d['func_import'], d['func_def'], d['func_comment'], d['func_impl'], d['func_whole'], d['func_test']

def write_code(path, idx, f_name, f_import, f_def, f_comment, f_impl, f_whole, f_test):
    def write_block(b, prefix=''):
        final_block = ""
        for l in b.split("\n"):
            final_block += prefix + l + "\n"
            
        return final_block

    with open(f"{path}/{idx}_{f_name}.py", 'w') as fw:
        fw.write(write_block(f_whole))
        
    with open(f"{path}/{idx}_{f_name}_test.py", 'w') as fw:
        fw.write(write_block(f_test))

code = get_code(std_block)
code
# write_code("output", 63, f_name, f_import, f_def, f_comment, f_impl, f_whole, f_test)

('classifier',
 'from transformers import pipeline',
 'def classifier(text: str) -> List[Dict[str, Union[str, float]]]:',
 'Performs sentiment analysis on the input text.\n\nArgs:\n    text (str): The input text to analyze.\n\nReturns:\n    List[Dict[str, Union[str, float]]]: A list of dictionaries containing the sentiment label and score.\n',
 "result = pipeline('sentiment-analysis')(text)\nreturn result",
 'from transformers import pipeline\n\ndef classifier(text: str) -> List[Dict[str, Union[str, float]]]:\n    """Performs sentiment analysis on the input text.\n\n    Args:\n        text (str): The input text to analyze.\n\n    Returns:\n        List[Dict[str, Union[str, float]]]: A list of dictionaries containing the sentiment label and score.\n    """\n    result = pipeline(\'sentiment-analysis\')(text)\n    return result',
 "def test_classifier():\n    assert classifier('We are very happy to show you the 🤗 Transformers library.') == [{'label': 'POSITIVE', 'score': 0.9998}]\n    as

In [84]:
import traceback

for idx, block in enumerate(all_valid_blocks):
    if idx + 1 <= 166:
        continue
    print(idx + 1, end="...")
    try:
        std_block = valid_block_to_std_block("".join(block))

        f_name, f_import, f_def, f_comment, f_impl, f_whole, f_test = get_code(std_block)

        formatted_number = str(idx + 1).zfill(5)
        
        write_code("output", formatted_number, f_name, f_import, f_def, f_comment, f_impl, f_whole, f_test)
    except Exception:
        print(block)
        traceback.print_exc()
        print(std_block)

167...['\n', 'If you belong to an organization and want to push your model under the organization name instead, just add it to the `repo_id`:\n', '\n', '```py\n', '>>> pt_model.push_to_hub("my-awesome-org/my-awesome-model")\n', '```\n']
```python
{
	"func_name": "push_to_hub",
	"func_import": "import transformers",
	"func_def": "def push_to_hub(repo_id: str) -> None:",
	"func_comment": "Pushes the model to the Hugging Face Model Hub under the given repository ID.\n\n:param repo_id: The ID of the repository in the format 'username/model-name'\n:return: None",
	"func_impl": "def push_to_hub(repo_id: str) -> None:\n    model = transformers.AutoModel.from_pretrained('model_name')\n    model.save_pretrained(repo_id)\n",
	"func_whole": "import transformers\n\ndef push_to_hub(repo_id: str) -> None:\n    \n    \"\"\"\n    Pushes the model to the Hugging Face Model Hub under the given repository ID.\n    \n    :param repo_id: The ID of the repository in the format 'username/model-name'\n    :re

Traceback (most recent call last):
  File "/tmp/ipykernel_768/4217252660.py", line 10, in <cell line: 3>
    f_name, f_import, f_def, f_comment, f_impl, f_whole, f_test = get_code(std_block)
  File "/tmp/ipykernel_768/2117595317.py", line 5, in get_code
    d = eval(json_data)
  File "<string>", line 1
    ```python
    ^
SyntaxError: invalid syntax


['\n', 'The `push_to_hub` function can also be used to add other files to a model repository. For example, add a tokenizer to a model repository:\n', '\n', '```py\n', '>>> tokenizer.push_to_hub("my-awesome-model")\n', '```\n']
```python
{
	"func_name": "push_to_hub",
	"func_import": "from transformers import PreTrainedModel",
	"func_def": "def push_to_hub(self, repo_name: str, repo_url: Optional[str] = None, organization: Optional[str] = None, use_auth_token: Optional[Union[str, bool]] = None, commit_message: Optional[str] = None, private: bool = False, **kwargs) -> str:",
	"func_comment": "Push the model repository to the Hugging Face Hub.\n\n        Args:\n            repo_name (:obj:`str`): The name of the repository to create or push to.\n            repo_url (:obj:`str`, `optional`): The URL of the repository to push to. If not provided, a new repository will be created.\n            organization (:obj:`str`, `optional`): The organization to push the repository to. If not provided

Traceback (most recent call last):
  File "/tmp/ipykernel_768/4217252660.py", line 10, in <cell line: 3>
    f_name, f_import, f_def, f_comment, f_impl, f_whole, f_test = get_code(std_block)
  File "/tmp/ipykernel_768/2117595317.py", line 5, in get_code
    d = eval(json_data)
  File "<string>", line 1
    ```python
    ^
SyntaxError: invalid syntax


170...171...172...['| **Input**                                                                                                               | **Output**                                   |\n', '|-------------------------------------------------------------------------------------------------------------------------|----------------------------------------------|\n', '| A beaver is swimming in the water | <audio controls><source src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tts_example.wav" type="audio/wav"> your browser does not support the audio element. </audio>\n', '\n', '---\n', '\n', '```py\n', 'agent.run(\n', '    "In the following `document`, where will the TRRF Scientific Advisory Council Meeting take place?",\n', '    document=document,\n', ')\n', '```\n']
```json
{
    "func_name": "agent.run",
    "func_import": "import agent",
    "func_def": "def run(question, document):",
    "func_comment": "Run the agent to answer the 

...

Traceback (most recent call last):
  File "/tmp/ipykernel_768/4217252660.py", line 10, in <cell line: 3>
    f_name, f_import, f_def, f_comment, f_impl, f_whole, f_test = get_code(std_block)
  File "/tmp/ipykernel_768/2117595317.py", line 5, in get_code
    d = eval(json_data)
  File "<string>", line 1
    ```python
    ^
SyntaxError: invalid syntax


['\n', 'Note that [`TapasTokenizer`] expects the data of the table to be **text-only**. You can use `.astype(str)` on a dataframe to turn it into text-only data.\n', 'Of course, this only shows how to encode a single training example. It is advised to create a dataloader to iterate over batches:\n', '\n', '```py\n', '>>> import tensorflow as tf\n', '>>> import pandas as pd\n', '\n', '>>> tsv_path = "your_path_to_the_tsv_file"\n', '>>> table_csv_path = "your_path_to_a_directory_containing_all_csv_files"\n', '\n', '\n', '>>> class TableDataset:\n', '...     def __init__(self, data, tokenizer):\n', '...         self.data = data\n', '...         self.tokenizer = tokenizer\n', '\n', '...     def __iter__(self):\n', '...         for idx in range(self.__len__()):\n', '...             item = self.data.iloc[idx]\n', '...             table = pd.read_csv(table_csv_path + item.table_file).astype(\n', '...                 str\n', '...             )  # be sure to make your table data text only\n', '

Traceback (most recent call last):
  File "/tmp/ipykernel_768/4217252660.py", line 10, in <cell line: 3>
    f_name, f_import, f_def, f_comment, f_impl, f_whole, f_test = get_code(std_block)
  File "/tmp/ipykernel_768/2117595317.py", line 5, in get_code
    d = eval(json_data)
  File "<string>", line 1
    ```json
    ^
SyntaxError: invalid syntax


894...895...896...['</pt>\n', '<tf>\n', 'Here we explain how you can use [`TFTapasForQuestionAnswering`] for inference (i.e. making predictions on new data). For inference, only `input_ids`, `attention_mask` and `token_type_ids` (which you can obtain using [`TapasTokenizer`]) have to be provided to the model to obtain the logits. Next, you can use the handy [`~models.tapas.tokenization_tapas.convert_logits_to_predictions`] method to convert these into predicted coordinates and optional aggregation indices.\n', '\n', "However, note that inference is **different** depending on whether or not the setup is conversational. In a non-conversational set-up, inference can be done in parallel on all table-question pairs of a batch. Here's an example of that:\n", '\n', '```py\n', '>>> from transformers import TapasTokenizer, TFTapasForQuestionAnswering\n', '>>> import pandas as pd\n', '\n', '>>> model_name = "google/tapas-base-finetuned-wtq"\n', '>>> model = TFTapasForQuestionAnswering.from_pretr

Traceback (most recent call last):
  File "/tmp/ipykernel_768/4217252660.py", line 10, in <cell line: 3>
    f_name, f_import, f_def, f_comment, f_impl, f_whole, f_test = get_code(std_block)
  File "/tmp/ipykernel_768/2117595317.py", line 8, in get_code
    return d['func_name'], d['func_import'], d['func_def'], d['func_comment'], d['func_impl'], d['func_whole'], d['func_test']
TypeError: string indices must be integers


In [4]:
# change func name
import os

def rename_files_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        # Check if the file is a regular file (not a directory)
        if os.path.isfile(os.path.join(directory_path, filename)):
            # Modify the filename as needed
            new_filename = "f" + filename
            # Construct the new file path
            new_file_path = os.path.join(directory_path, new_filename)
            # Rename the file
            os.rename(os.path.join(directory_path, filename), new_file_path)
            print(f"Renamed {filename} to {new_filename}")

# Example usage: specify the directory path you want to process
directory_path = "./output"
rename_files_in_directory(directory_path)

Renamed 00080_pad_sequences.py to f00080_pad_sequences.py
Renamed 00004_pipeline.py to f00004_pipeline.py
Renamed 00004_pipeline_test.py to f00004_pipeline_test.py
Renamed 00006_preprocess_dataset.py to f00006_preprocess_dataset.py
Renamed 00006_preprocess_dataset_test.py to f00006_preprocess_dataset_test.py
Renamed 00007_speech_recognizer.py to f00007_speech_recognizer.py
Renamed 00007_speech_recognizer_test.py to f00007_speech_recognizer_test.py
Renamed 00009_load_model_and_tokenizer.py to f00009_load_model_and_tokenizer.py
Renamed 00009_load_model_and_tokenizer_test.py to f00009_load_model_and_tokenizer_test.py
Renamed 00010_load_model.py to f00010_load_model.py
Renamed 00010_load_model_test.py to f00010_load_model_test.py
Renamed 00011_classify_french_text.py to f00011_classify_french_text.py
Renamed 00011_classify_french_text_test.py to f00011_classify_french_text_test.py
Renamed 00010_load_model_and_tokenizer.py to f00010_load_model_and_tokenizer.py
Renamed 00010_load_model_and_t

In [24]:
import glob

# Get a list of all .err files in the output directory
err_files = glob.glob('output/*.err')

c = 0
correct = 0
# Iterate through each .err file
for file_path in err_files:
    c += 1
    # print(f'Checking file: {file_path}')
    with open(file_path, 'r') as file:
        traceback_error_found = False
        # Iterate through each line in the file
        for line in file:
            # Check if the line contains 'Traceback'
            if 'Traceback' in line:
                traceback_error_found = True
                # print('Traceback error found:', line)
                break  # Exit the loop if a traceback error is found in the file
        # If no traceback error was found in the file
        if not traceback_error_found:
            print(f'No traceback error in this file. {file_path}')
            correct += 1
            
            # .py file
            print(file_path[:-9] + ".py")
            python_file = file_path[:-9] + ".py"
            with open(python_file, 'r') as f:
                for line in f:
                    print(line)
                    
            print("----")
            
print(c, correct, correct/c, 539*correct/c)

No traceback error in this file. output/f00010_load_model_and_tokenizer_test.err
output/f00010_load_model_and_tokenizer.py
from typing import *

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification



def load_model_and_tokenizer(model_name: str):

    model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return model, tokenizer

----
No traceback error in this file. output/f00010_load_model_test.err
output/f00010_load_model.py
from typing import *

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification



def load_model(model_name):

    model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return model, tokenizer

----
No traceback error in this file. output/f00040_train_model_test.err
output/f00040_train_model.py
from typing import *

from transformers import TFAutoModelForS